# Developing the RAG model 

This script will load the FAA Engine Maintenance manual, chunk them and store them in a vector database for retrieval 

In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader

api_key = os.environ["OPENAI_API_KEY"]



### Step 1: Loading the documents 

In [8]:
loader = PyPDFLoader("docs/FAA_Engine-Maintainance.pdf")

docs = loader.load()

In [12]:
docs[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.1 (Windows)', 'creationdate': '2023-07-11T10:37:28-05:00', 'author': 'FAA', 'moddate': '2023-07-13T09:49:40-04:00', 'title': 'AMT Handbook - Powerplant (FAA-H-8083-32B)', 'source': 'docs/FAA_Engine-Maintainance.pdf', 'total_pages': 65, 'page': 0, 'page_label': '1'}, page_content='10-1\nReciprocating Engine Overhaul\nBoth maintenance and complete engine overhauls are \nperformed normally at specified intervals. This interval is \nusually governed by the number of hours the powerplant \nhas been in operation. The actual overhaul period for a \nspecific engine is generally determined by the manufacturer’s \nrecommendations. Each engine manufacturer sets a total \ntime in service when the engine should be removed from \nservice and overhauled. Depending upon how the engine \nis used in service, the overhaul time can be mandatory. The \noverhaul time is listed in hours and is referred to as time \nbefo

### Step 2 : Splitting

We chose recursive character text splitting because we wanted to preserve the chunks in a semantical meaning. Chunk size is selected to be 800 As a rule of thumb for technical documents with an overlap of 15-20% which is around 150

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

rc_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=800,
    chunk_overlap=150
)

splitted_docs = rc_splitter.split_documents(docs)

In [ ]:
# print(len(splitted_docs[0].page_content))
# print(len(splitted_docs[1].page_content))
# print(splitted_docs[0].page_content)
# print(splitted_docs[1].page_content)


790
770


### Step 3 : Storage and retrieval 
